In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data_loader import load_and_align_prices
from src.validation import run_walk_forward
from src.monte_carlo import monte_carlo_simulation



In [3]:
prices = load_and_align_prices(
    "../data/gold.csv",
    "../data/silver.csv"
)


In [ ]:
with open("../results/statistical_summary.json", "r") as f:
    stats = json.load(f)

hedge_ratio = stats["hedge_ratio"]
half_life = int(stats["half_life_days"])

hedge_ratio, half_life

In [ ]:
wf_results = run_walk_forward(
    prices=prices,
    hedge_ratio=hedge_ratio,
    z_window=half_life,
    entry_z=2.0,
    exit_z=0.5,
    max_z=3.0,
    transaction_cost=0.0001,
    periods_per_year=252
)

wf_results


In [ ]:
in_sharpe = wf_results["in_sample"]["metrics"]["Sharpe Ratio"]
out_sharpe = wf_results["out_of_sample"]["metrics"]["Sharpe Ratio"]

in_sharpe, out_sharpe


In [ ]:
oos_returns = wf_results["out_of_sample"]["returns"]

mc = monte_carlo_simulation(
    oos_returns,
    num_simulations=1000
)


In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(mc["final_returns"], bins=50)
plt.title("Monte Carlo Distribution of Final Returns (OOS)")
plt.show()


In [ ]:
drawdowns = mc["max_drawdowns"]

prob_dd_20 = (drawdowns < -0.20).mean()
prob_dd_20
